In [1]:
import os
import nltk
import spacy
import es_core_news_sm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# Inicializar el lematizador de spaCy
nlp = spacy.load('es_core_news_sm')

# Definir el directorio donde están los archivos .txt
directorio = 'DiscursosOriginales/'  # Reemplaza esto con tu directorio

# Lista para almacenar los contenidos de los archivos
contenidos = []

# Leer los textos y combinarlos en un solo corpus
for filename in os.listdir(directorio):
    if filename.endswith('.txt'):
        with open(os.path.join(directorio, filename), 'r', encoding='utf-8') as file:
            contenido = file.read()
            print(f'Archivo {filename} leído')

            # Tokenizar y lematizar el contenido
            tokens = word_tokenize(contenido.lower())# Convertir a minúsculas
            tokens_sin_stopwords = [word for word in tokens if word not in STOP_WORDS]
            tokens_sin_stopwords = ' '.join(tokens_sin_stopwords)

            # Convertir los lemas nuevamente a texto
            contenidos.append(tokens_sin_stopwords)


Archivo 91265.txt leído
Archivo 83107.txt leído
Archivo 79915.txt leído
Archivo 153144.txt leído
Archivo 94447.txt leído
Archivo 84898.txt leído
Archivo 149691.txt leído
Archivo 89628.txt leído
Archivo 85012.txt leído
Archivo 85006.txt leído
Archivo 80142.txt leído
Archivo 96720.txt leído
Archivo 75444.txt leído
Archivo 86296.txt leído
Archivo 91891.txt leído
Archivo 164307.txt leído
Archivo 100454.txt leído
Archivo 80552.txt leído
Archivo 136643.txt leído
Archivo 84907.txt leído
Archivo 102280.txt leído
Archivo 101951.txt leído
Archivo 71541.txt leído
Archivo 90595.txt leído
Archivo 76776.txt leído
Archivo 96872.txt leído
Archivo 96866.txt leído
Archivo 85952.txt leído
Archivo 164266.txt leído
Archivo 94863.txt leído
Archivo 73020.txt leído
Archivo 101415.txt leído
Archivo 80396.txt leído
Archivo 153186.txt leído
Archivo 75121.txt leído
Archivo 100509.txt leído
Archivo 136905.txt leído
Archivo 86042.txt leído
Archivo 163938.txt leído
Archivo 81705.txt leído
Archivo 102450.txt leído
Ar

In [12]:
print(contenidos[6])

pandemia coronavirus amenaza salud compatriotas vida grupos riesgo , adultos mayores enfermos crónicos . afectando ingresos empleos chilenos poniendo riesgo supervivencia micro , pequeñas medianas empresas . prioridades compromisos : proteger salud vida compatriotas . asegurar abastecimiento bienes servicios esenciales , alimentos medicamentos , servicios utilizad pública , electricidad , agua telecomunicaciones . proteger empleos trabajadores ingresos familias . apoyar emprendedores , pymes empresas general puedan superar emergencia . plenamente conscientes temores , angustias problemas pandemia , impacto calidad vida , empleos ingresos , causando familias chilenas . plenamente conscientes dificultades enfrentando emprendedores pequeños empresarios , mantener marcha proyectos , emprendimientos empresas , sobrevivir crisis . razones , comienzos mes enero , preparando cumplir eficacia prioridades compromisos mencionados anteriormente . 19 marzo gobierno puso marcha plan emergencia , 12 

In [13]:
model = Word2Vec(tokens, vector_size=300, window=3, min_count=1, sg=0)

In [5]:
vectores_por_texto = []
for texto_tokens in contenidos:
    vectores_por_token = [model.wv[token] for token in texto_tokens if token in model.wv]
    if vectores_por_token:
        # Promediar los vectores para obtener un vector representativo del texto
        vector_promedio = sum(vectores_por_token) / len(vectores_por_token)
        vectores_por_texto.append(vector_promedio)
    else:
        # Si no hay tokens en el modelo, agregar un vector de ceros
        vectores_por_texto.append([0] * model.vector_size)

In [6]:
type(vectores_por_texto)

list

In [18]:
consulta = str(input("Ingrese su consulta: "))
tokens_consulta = word_tokenize(consulta.lower())
tokens_sin_stopwords = [word for word in tokens_consulta if word not in STOP_WORDS]
doc = ' '.join(tokens_sin_stopwords)
embedding_consulta = [model.wv[token] for token in doc if token in model.wv]
vector_promedio = sum(embedding_consulta) / len(embedding_consulta)
embedding_consulta = vector_promedio.reshape(1, -1)


In [19]:
embedding_consulta

array([[ 0.01023388,  0.05210041, -0.01638626,  0.03910257, -0.03224916,
        -0.06757865,  0.02971276,  0.11876783, -0.02035419, -0.0345153 ,
         0.07877095, -0.06861901,  0.00120774,  0.10611026, -0.02739983,
        -0.06003681,  0.08365318,  0.00266277,  0.02612573, -0.0301734 ,
         0.01457852, -0.01371299,  0.00888716,  0.06488197, -0.02403623,
        -0.00376186, -0.06127237,  0.05768761, -0.0229358 , -0.06422007,
         0.03212967, -0.04429083,  0.056705  , -0.03174298, -0.03436796,
         0.01886276,  0.05141894, -0.05666333, -0.03716008, -0.01176792,
        -0.04711466,  0.00058186,  0.00997148, -0.030861  ,  0.02209342,
         0.03854674, -0.0505456 , -0.02178939,  0.02482387,  0.01647235,
        -0.02825783, -0.00833007, -0.08128503,  0.03277436,  0.02627661,
        -0.03478461,  0.01269659, -0.05077127,  0.05634639, -0.00405626,
        -0.00834746,  0.00534229,  0.01309266,  0.0402784 , -0.02658059,
         0.04513637,  0.0030306 ,  0.07402444, -0.0

In [20]:
similitudes = []
counter = 0
for vector in vectores_por_texto:
    vector = vector.reshape(1, -1)
    similitud = cosine_similarity(embedding_consulta, vector)[0][0]
    similitudes.append((contenidos[counter], similitud))
    counter += 1

# Ordenar por similitud descendente
similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

In [21]:
for texto, similitud in similitudes[:5]:  # Muestra los 5 textos más relevantes
    print(f'Similitud: {similitud}')
    print(texto)
    print('-' * 30)

Similitud: 0.9999990463256836
: alegatos escuchado , concluir forma clara : , posición boliviana demostrado fragilidad carencia sustento fundamentos , derecho internacional hechos , historia 113 años chile bolivia firmaron tratado 1904. fragilidad traducido conducta permanentemente errática cambiante argumentación bolivia expresado proceso . , sentencia excepción preliminar corte internacional justicia , perfectamente obligación chile negociar resultado predeterminado , territorio , mar soberanía chilena , , bolivia derecho territorio mar país . , múltiples ocasiones , tratado chile bolivia , 1904 , válidamente celebrado , plenamente vigente fijó claridad , perpetuidad , límites chile bolivia . consecuencia , chile , país amante paz respetuoso derecho internacional tratados internacionales , cumplir , cumplir establecido tratado 1904 , constituye piedra angular regula relaciones pasadas , presentes futuras chile bolivia . tiempo bolivia deje confundir aspiraciones obligaciones , deje c